In [12]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are a SPARK SQL expert. Given an input question, create a syntactically correct SPARK SQL query to run. Unless otherwise specificed, do not return more than 3 rows.
Return only the SPARK QUERY without other words.
Only use the following tables: my_temp_table
Schema:|-- TransactionNo: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- ProductNo: string (nullable = true)
 |-- ProductName: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- CustomerNo: string (nullable = true)
 |-- Country: string (nullable = true)

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [17]:
from pyspark.sql import SparkSession
# Crea la SparkSession
spark = SparkSession.builder.appName("CSV to Spark SQL").getOrCreate()

ConnectionRefusedError: [Errno 61] Connection refused

In [3]:
csv_file_path = "/Users/danielepica/Desktop/Big_Data/progetto_personale/LangChain/Sales Transaction.csv"
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
#Per eseguire query SQL sul DataFrame, devi registrarlo come una vista temporanea.

# Registra il DataFrame come una vista temporanea
df.createOrReplaceTempView("my_temp_table")

In [4]:
#Eseguire Query SQL

#Ora puoi eseguire query SQL usando spark.sql(). La vista temporanea che hai creato (my_temp_table) può essere utilizzata nelle query.

# Esegui una query SQL
query = "SELECT count(TransactionNo) FROM my_temp_table LIMIT 10"
result_df = spark.sql(query)

# Mostra il risultato della query
result_df.show()

+--------------------+
|count(TransactionNo)|
+--------------------+
|              536350|
+--------------------+



In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.llms import Ollama

#base_url="https://0a8e-34-125-86-121.ngrok-free.app"

llm = Ollama(model = "llama3")

sql_chain = (
    RunnablePassthrough()  #assegna il valore corretto a schema che visualizzeremo nel prompt
    | prompt
    | llm
    | StrOutputParser()
)

In [6]:
user_question = 'Quanti giorni di vendite sono presenti nel dataset?'
nuova_stringa = sql_chain.invoke({"question": user_question})

# 'SELECT COUNT(*) AS TotalAlbums\nFROM Album;'

In [21]:
nuova_stringa

'SELECT COUNT(DISTINCT Date) AS DaysOfSales FROM Transaction;'

In [11]:
result_df = spark.sql(nuova_stringa)
result_df.show()

ConnectionRefusedError: [Errno 61] Connection refused